<a href="https://colab.research.google.com/github/shivg7706/Image-Captioning/blob/master/ImageCaptioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!cp ./drive/My\ Drive/Flickr8k_text.zip ./

In [0]:
!wget https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip
!unzip Flickr8k_Dataset.zip
# !unzip Flickr8k_text.zip
!rm -rf Flickr8k_Dataset.zip
# !rm -rf Flickr8k_text.zip

In [0]:
captions = open('Flickr8k.token.txt', 'r').read().strip().split('\n')

In [0]:
import numpy as np

In [0]:
d = {}
for caption in captions:
    caption = caption.split('\t')
    caption[0] = caption[0][:len(caption[0])-2]
    if caption[0] in d:
        d[caption[0]].append(caption[1])
    else:
        d[caption[0]] = [caption[1]]

In [0]:
d

In [0]:
from keras.preprocessing.image import img_to_array, load_img
from keras.applications.vgg16 import VGG16, preprocess_input

In [0]:

with open('Flickr_8k.trainImages.txt', 'r') as f:
    train_images = f.read().strip().split('\n')

In [0]:
with open('Flickr_8k.devImages.txt', 'r') as f:
    val_images = f.read().strip().split('\n')

In [0]:
with open('Flickr_8k.testImages.txt', 'r') as f:
    test_images = f.read().strip().split('\n')

In [26]:
vgg = VGG16(weights='imagenet')

W0715 09:06:01.101183 140372208797568 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0715 09:06:01.128064 140372208797568 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0715 09:06:01.136959 140372208797568 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0715 09:06:01.186632 140372208797568 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



553467904/553467096 [==============================] - 6s 0us/step


W0715 09:06:07.803430 140372208797568 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0715 09:06:07.805155 140372208797568 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



In [0]:
from keras.models import Model

inputs = vgg.input
output = vgg.layers[-2].output

model = Model(inputs, output)

In [28]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
def preprocessing(image):
    img = load_img('Flicker8k_Dataset/{}'.format(image), target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    
    return img

In [0]:
def get_encoding_images(image):
    img = preprocessing(image)
    pred = model.predict(img)
    pred = pred.reshape(4096)
    
    return pred

In [0]:
from tqdm import tqdm_notebook as tqdm

In [0]:
train_encode = {}

for image in tqdm(train_images):
    train_encode[image] = get_encoding_images(image)

In [0]:
val_encode = {}

for image in tqdm(val_images):
    val_encode[image] = get_encoding_images(image)

In [0]:
train_text = {}
for i in tqdm(train_images):
    train_text[i] = d[i]

In [0]:
val_text = {}
for i in tqdm(val_images):
    val_text[i] = d[i]

#### Adding _start_ and _end_ to each sentences

In [0]:
bhaji = []
for sublist in train_text.values():
    for sen in sublist:
        bhaji.append('<start> {} <end>'.format(sen))

In [0]:
word = []
for sen in bhaji:
    word += sen.split()

In [0]:
words = list(set(word))

In [68]:
len(words)

8256

In [0]:
word2idx = {val:index for index, val in enumerate(words)}
idx2word = {index:val for index, val in enumerate(words)}

In [71]:
idx2word[7706]

'brightly-lit'

In [72]:
word2idx['brightly-lit']

7706

In [0]:
import pandas as pd

In [80]:
df.head()

,0,1
0,2513260012_03d33305cf.jpg,[A black dog is running after a white dog in t...
1,2903617548_d3e38d7f88.jpg,"[A little baby plays croquet ., A little girl ..."
2,3338291921_fe7ae0c8f8.jpg,[A brown dog in the snow has something hot pin...
3,488416045_1c6d903fe0.jpg,"[A brown dog is running along a beach ., A bro..."
4,2644326817_8f45080b87.jpg,[A black and white dog with a red Frisbee stan...
